In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [2]:
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [3]:
merged_data = pd.merge(transactions, customers, on='CustomerID', how='left')

In [4]:
customer_profiles = merged_data.groupby('CustomerID').agg({
    'Quantity': 'sum',  # Total quantity purchased by the customer
    'TotalValue': 'sum',  # Total spending by the customer
    'Region': 'first'  # We'll encode Region as a categorical value later
}).reset_index()

In [5]:
customer_profiles['Region'] = customer_profiles['Region'].astype('category').cat.codes

In [6]:
X = customer_profiles[['Quantity', 'TotalValue', 'Region']].values

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
cosine_sim = cosine_similarity(X_scaled)

In [10]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_profiles['CustomerID'], columns=customer_profiles['CustomerID'])

In [11]:
lookalike_recommendations = []

In [12]:
for customer in customer_profiles['CustomerID'][:20]:
    similar_customers = cosine_sim_df[customer].nlargest(4)
    similar_customers = similar_customers.drop(customer)

In [13]:
recommended_customers = similar_customers.head(3)

In [16]:
lookalike_recommendations = []

for customer in customer_profiles['CustomerID'][:20]:
    similar_customers = cosine_sim_df[customer].nlargest(4)  
    similar_customers = similar_customers.drop(customer) 
    recommended_customers = similar_customers.head(3)
    recommendation = {
        'CustomerID': customer,
        'SimilarCustomerID1': recommended_customers.index[0],
        'Score1': recommended_customers.iloc[0],
        'SimilarCustomerID2': recommended_customers.index[1],
        'Score2': recommended_customers.iloc[1],
        'SimilarCustomerID3': recommended_customers.index[2],
        'Score3': recommended_customers.iloc[2]
    }
    lookalike_recommendations.append(recommendation)


In [17]:
lookalike_df = pd.DataFrame(lookalike_recommendations)

In [18]:
lookalike_df.to_csv('Sonal_Garg_Lookalike.csv', index=False)

In [19]:
print("Lookalike model recommendations have been saved to 'Sonal_Garg_Lookalike.csv'.")

Lookalike model recommendations have been saved to 'Sonal_Garg_Lookalike.csv'.
